In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Parametri
ticker_symbol = "^SPX"
target_expiration = "2025-08-08"  # Inserire una delle scadenze disponibili sotto
min_strength_pct = 0.01  # Mostra solo barre sopra al 1% del massimo
grouping_interval = 50 # Intervallo di raggruppamento per gli strike

# Download opzioni
ticker = yf.Ticker(ticker_symbol)
all_expirations = ticker.options

if target_expiration not in all_expirations:
    raise ValueError(f"La scadenza {target_expiration} non è disponibile. Disponibili: {all_expirations}")

opt_chain = ticker.option_chain(target_expiration)
calls = opt_chain.calls.copy()
puts = opt_chain.puts.copy()

# Calcolo forza relativa = open interest * premio
calls["strength"] = calls["openInterest"] * calls["lastPrice"]
puts["strength"] = puts["openInterest"] * puts["lastPrice"]

# Raggruppamento per intervallo (tutte le opzioni vengono raggruppate qui)
calls["strike_group"] = (calls["strike"] // grouping_interval) * grouping_interval
puts["strike_group"] = (puts["strike"] // grouping_interval) * grouping_interval

grouped_calls = calls.groupby("strike_group")["strength"].sum().reset_index()
grouped_puts = puts.groupby("strike_group")["strength"].sum().reset_index()

# Merge affiancato su strike group
merged_grouped = pd.merge(
    grouped_calls.rename(columns={"strength": "call_strength", "strike_group": "strike"}),
    grouped_puts.rename(columns={"strength": "put_strength", "strike_group": "strike"}),
    on="strike",
    how="outer"
).fillna(0).sort_values("strike")

# Calcola la forza totale del gruppo per il filtro
merged_grouped["total_strength"] = merged_grouped["call_strength"] + merged_grouped["put_strength"]

# Filtro dinamico: rimuove gruppi con forza totale sotto soglia relativa (APPLICATO DOPO IL RAGGRUPPAMENTO)
max_total_strength = merged_grouped["total_strength"].max()
merged_grouped_filtered = merged_grouped[merged_grouped["total_strength"] >= max_total_strength * min_strength_pct].copy()

# Plot - usa i dati filtrati per la visualizzazione
x = merged_grouped_filtered["strike"].astype(str)
call_heights = merged_grouped_filtered["call_strength"]
put_heights = merged_grouped_filtered["put_strength"]

bar_width = 0.4
x_indexes = np.arange(len(x)) # Use numpy for easier index handling

plt.figure(figsize=(14, 6))
plt.bar(x_indexes - bar_width/2, call_heights, width=bar_width, label="CALL", align="center")
plt.bar(x_indexes + bar_width/2, put_heights, width=bar_width, label="PUT", align="center")
plt.xticks(x_indexes, x, rotation=90)
plt.xlabel("Strike Price Group")
plt.ylabel("Relative Strength (OI * Premium)")
plt.title(f"{ticker_symbol} DPD - Exp: {target_expiration} (Grouped by {grouping_interval}, Filtered by Total Strength)")
plt.legend()
plt.tight_layout()
plt.grid(True)

# Add vertical line for current price - usa i dati filtrati per trovare l'indice più vicino
current_price = ticker.history(period="1d")["Close"].iloc[0]

# Find the index of the closest strike price group to the current price in the filtered data
closest_strike_group_index = np.abs(merged_grouped_filtered["strike"] - current_price).argmin()

# Use the index of the closest strike price group for the vertical line
plt.axvline(x=x_indexes[closest_strike_group_index], color='red', linestyle='--', label=f'Current Price: {current_price:.2f}')
plt.legend()

plt.show()

# Display the filtered grouped data table
display(merged_grouped_filtered)

